In [1]:
import time
import numpy as np
from Conways import*

pygame 2.0.2 (SDL 2.0.16, Python 3.9.4)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
env = conways(size = 10, max_it=1000)
print(env.run())

25


In [3]:
x = np.zeros(shape=(10, 10), dtype=bool)
x[1, 1] = True
x[1, 2] = True
x[2, 1] = True
x[2, 2] = True
env.reset(x, offset=False)
actions = env.run()
print(actions)

inf


In [4]:
alpha = 0.1 
max_iterations = 6000
gamma = 0.995

total = 0
tot_reward = 0
prev_reward = 0

q_table_row_limit = 75

epsilon = 1
epsilon_decay = 0.99995

In [5]:
q_table = np.random.uniform(low=0, high=1, size=[q_table_row_limit, 10, 10])
q_table.shape

print(np.argmax(q_table[0]))
print(np.max(q_table[0]))
print(q_table[0])

40
0.9870042661452071
[[0.14120214 0.22254572 0.80184494 0.71220061 0.6996153  0.21486675
  0.25740071 0.28097463 0.44266125 0.62307983]
 [0.62619222 0.63285864 0.45167726 0.60551346 0.80644274 0.53037967
  0.95879052 0.33095176 0.6682666  0.57628749]
 [0.44079342 0.43824752 0.28184903 0.26546194 0.56451571 0.14489895
  0.05574545 0.82246225 0.70362354 0.35224032]
 [0.90128381 0.33444899 0.94579565 0.4776696  0.50739331 0.05587027
  0.68633847 0.78820281 0.41725038 0.4826092 ]
 [0.98700427 0.8643478  0.49713513 0.26192417 0.85243923 0.1571941
  0.10249528 0.28914708 0.40045761 0.38076883]
 [0.73033652 0.51462103 0.63325364 0.88471974 0.98481937 0.45990494
  0.68045381 0.05273588 0.04302183 0.37263489]
 [0.7120521  0.6699277  0.53285174 0.27942962 0.9854651  0.45010288
  0.31119463 0.30121289 0.60995068 0.76335995]
 [0.36963178 0.89129016 0.86020199 0.18416986 0.84347936 0.42962415
  0.87853891 0.16639693 0.17834245 0.80447414]
 [0.79101311 0.50850261 0.58869565 0.53874125 0.7833547  0.

In [6]:

state_score = np.array([float('-inf') for i in range(q_table_row_limit)])
state_table = np.random.uniform(low=0, high=1, size=[q_table_row_limit, 10, 10]) 

def get_discrete_state(s):
    for i in range(q_table_row_limit):
        if np.array_equal(s, state_table[i]):
            return i
    i = np.argmin(state_score)
    state_table[i] = np.copy(s)
    q_table[i] = np.random.uniform(low=0, high=1, size=[10, 10])
    return i


In [7]:
current_state = np.zeros(shape=(10, 10), dtype=bool)

#take initial action
discrete_state = get_discrete_state(current_state)

action = np.argmax(q_table[discrete_state])
action = (int(action/10), action%10)
current_state[action] = True
best_state = current_state
env.reset(current_state, offset = False)
state_score[discrete_state] = env.run()
best_reward = float('-inf')

for i in range(max_iterations):
    t0 = time.time()
    

    if (i % 2000 == 0):
        print(f"Episode: {i}")

    reward = 0
    #test possible actions after taking that action
    for j in range(q_table_row_limit):
        if np.random.random() > epsilon:
            action = np.argmax(q_table[discrete_state])
        else:
            action = np.random.randint(0, 100)
        action = (int(action/10), action%10)
        current_state[action] = True
        env.reset(current_state, offset = False)

        new_discrete_state = get_discrete_state(current_state)

        state_score[discrete_state] = env.run()
        reward += state_score[discrete_state]

        if (state_score[discrete_state] > best_reward):
            best_state = np.copy(current_state)
            best_reward = state_score[discrete_state]

        max_future_q = np.max(q_table[new_discrete_state])
        current_q = q_table[(discrete_state, action)]

        new_q = (1 - alpha) * current_q + alpha * (state_score[discrete_state] + gamma * max_future_q)
        q_table[(discrete_state, action)] = new_q

        discrete_state = new_discrete_state
    
    
    current_state = np.copy(best_state)
    reward = np.max(state_score)
    if epsilon > 0.05: 
        if reward > prev_reward and i > 10000:
            epsilon = epsilon_decay ** (i - 10000)

            if i % 500 == 0:
                print("Epsilon: " + str(epsilon))
    
    t1 = time.time()
    iteration_time = t1-t0
    total += iteration_time

    tot_reward += reward
    prev_reward = reward

    if i % 1000 == 0: 
        mean = total / 1000
        print("Time Average: " + str(mean))
        total = 0

        mean_reward = tot_reward / 1000
        print("Mean Reward: " + str(mean_reward))
        tot_reward = 0

Episode: 0
Time Average: 0.00024299240112304687
Mean Reward: 0.064
Time Average: 0.46933180117607115
Mean Reward: 114.427
Episode: 2000
Time Average: 0.3529937403202057
Mean Reward: 121.0
Time Average: 0.24940313506126405
Mean Reward: 121.0
Episode: 4000
Time Average: 0.2457814302444458
Mean Reward: 121.0
Time Average: 0.2478071348667145
Mean Reward: 121.0


In [8]:
Final = best_state
print(Final)
env.reset(Final, offset = False)
env.render = True
print(env.run())

[[False  True  True False False False False False False False]
 [False False  True  True False False  True  True  True False]
 [ True False  True  True False False False False False False]
 [False False False False False False  True False False False]
 [False  True False  True False  True  True  True False False]
 [False False False  True  True False  True  True  True False]
 [ True False False False False False False False False  True]
 [ True False  True False False False False False False False]
 [False  True False  True  True False False  True False  True]
 [ True False False False False  True  True False  True False]]
121
